In [15]:
import asyncio
from playwright.async_api import async_playwright
import qrcode
from PIL import Image
import io
import base64
from pyzbar import pyzbar
import os

In [16]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
context = await browser.new_context()
page = await context.new_page()

In [17]:
await page.goto("https://web.whatsapp.com")

<Response url='https://web.whatsapp.com/' request=<Request url='https://web.whatsapp.com/' method='GET'>>

In [8]:
last_message = None
last_person = None

In [31]:
messages = await page.query_selector_all(
    '//*[@id="pane-side"]/div[2]/div/div/child::div'
)

clean_messages = []
for message in messages:
    _message = await message.inner_text()  # Use inner_text to get the text content
    # _message = _message.split("\n")
    clean_messages.append(_message)
    # if len(_message) == 2:
    #     clean_messages.append(
    #         {
    #             "sender": _message[0],
    #             "time": _message[1],
    #             "message": "",
    #             "unread": False,
    #             "no_of_unread": 0,
    #             "group": False,
    #         }
    #     )
    # elif len(_message) == 3:
    #     clean_messages.append(
    #         {
    #             "sender": _message[0],
    #             "time": _message[1],
    #             "message": _message[2],
    #             "unread": False,
    #             "no_of_unread": 0,
    #             "group": False,
    #         }
    #     )
    # elif len(_message) == 4:
    #     clean_messages.append(
    #         {
    #             "sender": _message[0],
    #             "time": _message[1],
    #             "message": _message[2],
    #             "unread": _message[-1].isdigit(),
    #             "no_of_unread": int(_message[-1]) if _message[-1].isdigit() else 0,
    #             "group": False,
    #         }
    #     )
    # elif len(_message) == 5:
    #     clean_messages.append(
    #         {
    #             "sender": _message[0],
    #             "time": _message[1],
    #             "message": "",
    #             "unread": _message[-1].isdigit(),
    #             "no_of_unread": int(_message[-1]) if _message[-1].isdigit() else 0,
    #             "group": True,
    #         }
    #     )
    # elif len(_message) == 6:
    #     clean_messages.append(
    #         {
    #             "sender": _message[0],
    #             "time": _message[1],
    #             "message": _message[4],
    #             "unread": _message[-1].isdigit(),
    #             "no_of_unread": int(_message[-1]) if _message[-1].isdigit() else 0,
    #             "group": True,
    #         }
    #     )
    # else:
    #     print(f"Unknown message format: {_message}")

clean_messages

['Kausthubh\n11:22 PM\nHey',
 'Amazon.in\n11:28 AM\nAmazon Great Indian Festival is LIVE now!  We’ve missed you at Amazon! Here’s a special Reward to welcome you back. Get ₹100 cashback on purchase above ₹500* Get Free Delivery on your next order!  Half Price Store (Minimum 50% off*) - https://www.amazon.in/wa_agifeg  Grand opening deals - https://www.amazon.in/wa_agifns Pay with UPI or Cash at the time of delivery Great Prices! Easy Returns* Hurry - Valid for limited time only!  Click here to shop now - https://www.amazon.in/wa_agifac  10% Instant Discount* with SBI Debit & Credit Card *T&C Apply, visit Amazon.in',
 "VISHNU!!!! prez\n11:39 AM\nIt's probably gonna be on November",
 'slaves of rakuten\n12:50 PM\nTASC-Pratham\n:\xa0\nSticker',
 'Nandan Pai\n1:15 PM\nsure! thanks :))',
 'AIML Official grp 2021-25\n1:39 PM\n~\u202fDr. Sharada Udaya Shenoy\n:\xa0\nPhoto',
 'Dsu dev hack\n5:08 PM\nAyush Chaudhary\n:\xa0\nTeam Name (Registered on Devfolio) Dream Machine Theme AI\xa0\xa0ML Uni

In [46]:
messages = await page.query_selector_all(
    'div[role="listitem"][style*="transform: translateY(72px)"]'
)

clean_messages = []
for message in messages:
    # Extract the height from the style attribute
    style = await message.get_attribute("style")
    height = None
    if style:
        # Example style: "z-index: 482; transition: transform 200ms ease-in-out; height: 72px; transform: translateY(72px);"
        for item in style.split(";"):
            if "transform" in item:
                if "translateY" in item:
                    transform_value = item.split("translateY(")[1].split("px")[0]
                    height = int(transform_value)
                    break
                break

    # Extract the name from the title attribute of the span
    name_span = await message.query_selector("span[title]")
    name = await name_span.get_attribute("title") if name_span else "Unknown"

    # Extract the time from the corresponding div
    time_div = await message.query_selector("div._ak8i")
    time = await time_div.inner_text() if time_div else "Unknown"

    # Extract the message text
    message_span = await message.query_selector("span.x78zum5 span")
    message_text = await message_span.inner_text() if message_span else ""

    # Append the extracted data to the list
    clean_messages.append(
        {"height": height, "name": name, "time": time, "message": message_text}
    )

clean_messages.sort(
    key=lambda x: x["height"] if x["height"] is not None else float("inf")
)
clean_messages

[{'height': None, 'name': 'Kausthubh', 'time': 'Yesterday', 'message': ''},
 {'height': None, 'name': 'Kausthubh', 'time': '11:22 PM', 'message': ''}]

In [53]:
async def fetch_specific_listitem():
    # Define a more specific CSS selector using regex to match the exact transform value
    selector = 'div[role="listitem"][style*="transform: translateY(72px)"]'

    # Alternatively, use a regex to ensure precise matching
    # Note: Playwright's CSS selectors don't support full regex, so XPath might be better for complex matches

    # Wait for the element to be present in the DOM
    try:
        specific_div = await page.wait_for_selector(selector, timeout=60000)
        if specific_div:
            print("Div with role='listitem' and transform: translateY(0px) found.")
            return specific_div
    except Exception as e:
        print(f"Error: {e}")

    print("Desired div not found.")
    return None


# Usage example
specific_div = await fetch_specific_listitem()
await specific_div.inner_text()

Div with role='listitem' and transform: translateY(0px) found.


'Kausthubh\nYesterday\nHey\nKausthubh\n11:22 PM\nHey'

In [58]:
from playwright.async_api import Page


async def fetch_specific_listitem(page: Page):
    # Define the CSS selector for divs with role="listitem"
    selector = 'div[role="listitem"]'

    # Wait for at least one such element to be present in the DOM
    await page.wait_for_selector(selector, timeout=60000)

    # Query all matching elements
    list_items = await page.query_selector_all(selector)

    for item in list_items:
        # Retrieve the 'transform' style property
        transform = await item.evaluate(
            "element => window.getComputedStyle(element).transform"
        )

        # Check if the transform includes translateY(72px)
        if transform:
            # The computed transform matrix for translateY(72px) is typically "matrix(1, 0, 0, 1, 0, 72)"
            if "matrix" in transform:
                # Parse the translateY value from the matrix
                parts = transform.replace("matrix(", "").replace(")", "").split(", ")
                if len(parts) == 6:
                    translate_y = float(parts[5])
                    if translate_y == 0:
                        print("Div with role='listitem' and translateY=72px found.")
                        return item
            elif "translateY(72px)" in transform:
                # In some cases, the transform might be expressed directly
                print("Div with role='listitem' and translateY=72px found.")
                return item

    print("Desired div not found.")
    return None


# Usage example
specific_div = await fetch_specific_listitem(page)
await specific_div.inner_text()

Div with role='listitem' and translateY=72px found.


'Kausthubh\n12:24 AM\nfuck\n1\nKausthubh\n11:22 PM\nHey'

In [59]:
async def search_person(name: str, message: str):
    search_input_selector = (
        "div[contenteditable='true'][role='textbox'][data-lexical-editor='true']"
    )
    await page.wait_for_selector(search_input_selector, timeout=60000)
    search_input = await page.query_selector(search_input_selector)

    if search_input:
        # Type the name into the search input
        await search_input.click()

        await page.wait_for_timeout(2000)

        # Clear all messages
        await page.keyboard.down("Control")
        await page.keyboard.press("A")
        await page.keyboard.up("Control")
        await page.keyboard.press("Backspace")

        await page.wait_for_timeout(1000)
        await search_input.type(name, delay=100)
        await page.keyboard.press("Enter")  # Use page.keyboard to press Enter

        # Wait for search results to load
        await page.wait_for_timeout(2000)

        # Clear all messages
        await page.keyboard.down("Control")
        await page.keyboard.press("A")
        await page.keyboard.up("Control")
        await page.keyboard.press("Backspace")

        # Wait for the chat to clear
        await page.wait_for_timeout(1000)
        # Type the message
        # Type the message
        await page.keyboard.type(message, delay=100)

        # Simulate pressing Enter
        await page.keyboard.press("Enter")

        # Wait a moment before pressing Enter
        await page.wait_for_timeout(500)

        # Press Enter to send the message
        await page.keyboard.press("Enter")

        # Wait for the message to be sent
        await page.wait_for_timeout(1000)

    else:
        print("Search input not found.")

In [70]:
await search_person("kausthubh", "this is a test message pls ignore")

In [67]:
while True:
    m = await get_list_of_messages()
    print(m[0]["sender"])

    if m[0]["unread"] == True:
        await search_person(
            m[0]["sender"],
            "This is a test message for my hackathon project please ignore",
        )
        break
    else:
        print("No unread messages.")

Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosli

CancelledError: 

In [ ]:
def decode_qr_from_image(image_bytes):
    try:
        image = Image.open(io.BytesIO(image_bytes))
        decoded_objects = pyzbar.decode(image)
        for obj in decoded_objects:
            return obj.data.decode("utf-8")
        return None
    except Exception as e:
        print(f"Failed to decode QR from image: {e}")
        return None


def display_qr_in_terminal(data):
    try:
        qr = qrcode.QRCode()
        qr.add_data(data)
        qr.make()
        qr.print_ascii(invert=True)
    except Exception as e:
        print(f"Failed to display QR in terminal: {e}")

In [ ]:
qrinfo = None

try:
    await page.wait_for_selector("div._akau", timeout=600)
    print("QR code container detected.")

    qr_div = await page.query_selector("div._akau")
    if qr_div:
        qr_info = await qr_div.get_attribute("data-ref")
        if qr_info:
            qrinfo = qr_info
    else:
        print("data-ref attribute not found. Proceeding to screenshot method.")
except Exception as e:
    print(f"Error while extracting data-ref: {e}")

In [ ]:
qrinfo

In [ ]:
display_qr_in_terminal(qrinfo)

In [ ]:
screenshot = await page.screenshot()
qr_info = decode_qr_from_image(screenshot)
if qr_info:
    return qr_info
else:
    print("Failed to decode QR code from screenshot.")
    return None

In [ ]:
async def extract_login_qr(self):
    assert page != None

In [ ]:
try:
    # Wait for the search input to be visible using a more stable selector
    search_input_selector = (
        "div[contenteditable='true'][role='textbox'][data-lexical-editor='true']"
    )
    await page.wait_for_selector(search_input_selector, timeout=60000)
    search_input = await page.query_selector(search_input_selector)

    if search_input:
        # Type the name 'kausthubh' into the search input
        await search_input.click()
        await search_input.type(
            "kausthubh", delay=100
        )  # Adding a slight delay can help with typing simulation

        # Wait for search results to load
        await page.wait_for_timeout(2000)

        # Wait for the specific search result to be visible and click on it
        search_result_selector = "p.selectable-text[title='kausthubh']"
        search_result = await page.wait_for_selector(
            search_result_selector, timeout=60000
        )

        if search_result:
            await search_result.click()
            print("Chat with 'kausthubh' opened.")
        else:
            print(
                "Search result for 'kausthubh' not found. Clicking on the first chat."
            )

            # Selector for the first chat in the chat list
            first_chat_selector = (
                "div.x1y332i5:nth-child(3)"  # Update this selector based on actual DOM
            )
            first_chat = await page.wait_for_selector(
                first_chat_selector, timeout=60000
            )

            if first_chat:
                await first_chat.click()
                print("First chat clicked.")
            else:
                print("Failed to find the first chat.")
    else:
        print("Search input not found.")
except Exception as e:
    print(f"Error while searching for 'kausthubh': {e}")

## Closing the browser and saving the context

In [ ]:
await context.storage_state(path="browser_context.json")

if browser:
    await browser.close()
elif context:
    await context.close()

await playwright.stop()

In [ ]:
page2 = await browser.new_page()